# MonteCarlo Control

Control is the process of finding the best policy for a given environment. We follow generalised policy iteration and alternate between policy evaluation and policy improvement.


In [1]:
import numpy as np
from gridworld import GridWorld
np.set_printoptions(precision=3,suppress=True)

Return computation function now takes as input a list containing a 3-tuple (state,action,reward) and the discount factor gamma, the output is a value representing the return from that state.

In [2]:
def get_return(state_list, gamma):
    counter = 0
    return_value = 0
    for visit in state_list:
        reward = visit[2]
        return_value += reward * np.power(gamma, counter)
        counter += 1
    return return_value

Get a policy and improve it by taking a greedy action.

In [3]:
def improve_policy(episode_list, policy_matrix, state_action_matrix):
    
    for visit in episode_list:
        observation = visit[0]
        col = observation[1] + (observation[0]*4)
        if(policy_matrix[observation[0], observation[1]] != -1):      
            policy_matrix[observation[0], observation[1]] = \
                np.argmax(state_action_matrix[:,col])
    return policy_matrix


1.Lets create a grid world, the states marked 1 are terminal state and those marked -1 contain obstacles. <br>
2.The agent receives a reward of -0.04 for every move from non-terminal states <br>
3.The  actions are UP(0), RIGHT(1), DOWN(2) and LEFT(3)

In [4]:

env = GridWorld(3, 4)

#Define the state matrix
state_matrix = np.zeros((3,4))
state_matrix[0, 3] = 1
state_matrix[1, 3] = 1
state_matrix[1, 1] = -1
print("State Matrix:")
print(state_matrix)

#Define the reward matrix
reward_matrix = np.full((3,4), -0.04)
reward_matrix[0, 3] = 1
reward_matrix[1, 3] = -1
print("Reward Matrix:")
print(reward_matrix)

#Define the transition matrix
transition_matrix = np.array([[0.8, 0.1, 0.0, 0.1],
                              [0.1, 0.8, 0.1, 0.0],
                              [0.0, 0.1, 0.8, 0.1],
                              [0.1, 0.0, 0.1, 0.8]])

#Random policy
policy_matrix = np.random.randint(low=0, high=4, size=(3, 4)).astype(np.float32)
policy_matrix[1,1] = np.NaN #NaN for the obstacle at (1,1)
policy_matrix[0,3] = policy_matrix[1,3] = -1 #No action for the terminal states

#Set the matrices in the world
env.setStateMatrix(state_matrix)
env.setRewardMatrix(reward_matrix)
env.setTransitionMatrix(transition_matrix)

State Matrix:
[[ 0.  0.  0.  1.]
 [ 0. -1.  0.  1.]
 [ 0.  0.  0.  0.]]
Reward Matrix:
[[-0.04 -0.04 -0.04  1.  ]
 [-0.04 -0.04 -0.04 -1.  ]
 [-0.04 -0.04 -0.04 -0.04]]


In [5]:

state_action_matrix = np.random.random_sample((4,12)) # Q
#init with 1.0e-10 to avoid division by zero
running_mean_matrix = np.full((4,12), 1.0e-10) 
gamma = 0.999
tot_epoch = 1000
print_epoch = 20

for epoch in range(tot_epoch):
    #Starting a new episode
    episode_list = list()
    #Reset and return the first observation and reward
    observation = env.reset(exploring_starts=True)
    #action = np.random.choice(4, 1)
    #action = policy_matrix[observation[0], observation[1]]
    #episode_list.append((observation, action, reward))
    is_starting = True
    for _ in range(1000):
        #Take the action from the action matrix
        action = policy_matrix[observation[0], observation[1]]
        #If the episode just started then it is
            #necessary to choose a random action (exploring starts)
        if(is_starting): 
            action = np.random.randint(0, 4)
            is_starting = False      
        #Move one step in the environment and get obs and reward
        new_observation, reward, done = env.step(action)
        #Append the visit in the episode list
        episode_list.append((observation, action, reward))
        observation = new_observation
        if done: break
    #The episode is finished, now estimating the utilities
    counter = 0
    #Checkup to identify if it is the first visit to a state
    checkup_matrix = np.zeros((4,12))
    #This cycle is the implementation of First-Visit MC.
    #For each state stored in the episode list check if it
    #is the rist visit and then estimate the return.
    for visit in episode_list:
        observation = visit[0]
        action = visit[1]
        col = int(observation[1] + (observation[0]*4))
        row = int(action)
        if(checkup_matrix[row, col] == 0):
            return_value = get_return(episode_list[counter:], gamma)
            running_mean_matrix[row, col] += 1
            state_action_matrix[row, col] += return_value
            checkup_matrix[row, col] = 1
        counter += 1
    #Policy Update
    policy_matrix = improve_policy(episode_list, 
                                  policy_matrix, 
                                  state_action_matrix/running_mean_matrix)
    #Printing
    if(epoch % print_epoch == 0):
        print("")
        #print("State-Action matrix after " + str(epoch+1) + " iterations:") 
        #print(state_action_matrix / running_mean_matrix)
        print("Policy matrix after " + str(epoch+1) + " iterations:") 
        print(policy_matrix)
        
#Time to check the value  matrix obtained
print("value matrix after " + str(tot_epoch) + " iterations:")
print(state_action_matrix / running_mean_matrix)

print("Policy matrix after " + str(epoch+1) + " iterations:") 
print(policy_matrix)



Policy matrix after 1 iterations:
[[ 1.  0.  3. -1.]
 [ 1. nan  1. -1.]
 [ 2.  3.  0.  3.]]

Policy matrix after 21 iterations:
[[ 2.  1.  1. -1.]
 [ 2. nan  0. -1.]
 [ 0.  2.  0.  0.]]

Policy matrix after 41 iterations:
[[ 2.  1.  1. -1.]
 [ 2. nan  0. -1.]
 [ 1.  2.  0.  0.]]

Policy matrix after 61 iterations:
[[ 2.  2.  1. -1.]
 [ 2. nan  0. -1.]
 [ 1.  2.  0.  0.]]

Policy matrix after 81 iterations:
[[ 2.  2.  1. -1.]
 [ 2. nan  0. -1.]
 [ 1.  1.  0.  0.]]

Policy matrix after 101 iterations:
[[ 2.  2.  1. -1.]
 [ 2. nan  0. -1.]
 [ 1.  1.  0.  0.]]

Policy matrix after 121 iterations:
[[ 2.  1.  1. -1.]
 [ 2. nan  0. -1.]
 [ 1.  1.  0.  0.]]

Policy matrix after 141 iterations:
[[ 2.  1.  1. -1.]
 [ 2. nan  0. -1.]
 [ 1.  1.  0.  0.]]

Policy matrix after 161 iterations:
[[ 2.  1.  1. -1.]
 [ 2. nan  0. -1.]
 [ 1.  1.  0.  0.]]

Policy matrix after 181 iterations:
[[ 2.  1.  1. -1.]
 [ 2. nan  0. -1.]
 [ 1.  1.  0.  0.]]

Policy matrix after 201 iterations:
[[ 2.  1.  1. -1.]
